In [4]:
import numpy as np
import os
import sys
dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories
import math

In [5]:
# method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
method = METHOD_SCALAR_QUANTIZE
user_method = "METHOD_USER_SCALAR_QUANTIZE"
# user_method = "METHOD_USER_QUANTIZE"


In [6]:
feedback_bits = 256
ncodebooks=64
ncentroids=256
train_sam_num = 3000 # 训练集样本数
qbits = 8
prop_A = 0
prop_B = 0

In [7]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'

# dir_result = os.path.join(dir_result, method)
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, "fc1")
try:
    os.mkdir(dir_result)
except FileExistsError:
    pass 

In [8]:
# from turtle import right


def saturation_sort(A, proportion = 0.3, axis = 0): #饱和排序：proportion:饱和比例（一般为30%）
    axis_quantity = A.shape[axis] # axis = 0返回行数，1返回列数
    
    mins_ori = A.min(axis=axis, keepdims=True)
    # A_offset = A - offsets
    ranges_ori = (A - mins_ori).max(axis=axis, keepdims=True) + 1e-20
    if proportion == 0:
        return mins_ori, ranges_ori
    elif proportion > 0 and proportion <= 1:
        ix_threshold = math.floor(proportion*axis_quantity) # 前proportion%的最后一位
        Ato0 = A - ranges_ori/2. - mins_ori # 使A的axis维的最大最小值关于0对称
        Ato0abs = np.absolute(Ato0)
        Ato0abssort = np.sort(Ato0abs, axis=axis)
        if axis == 0:
            threshold = Ato0abssort[np.ix_([axis_quantity - ix_threshold], range(Ato0abssort.shape[1]))]
        elif axis == 1:
            threshold = Ato0abssort[np.ix_(range(Ato0abssort.shape[0]), [axis_quantity - ix_threshold])]
        mins_new = ranges_ori/2. + mins_ori - threshold
        ranges_new = 2 * threshold
        # print('ranges_ori')
        # print(ranges_ori)
        # print('mins_ori')
        # print(mins_ori)
        # print('Ato0')
        # print(Ato0)
        # print('Ato0abs')
        # print(Ato0abs)
        # print('Ato0abssort')
        # print(Ato0abssort)
        # print('threshold')
        # print(threshold)
        return mins_new, ranges_new


In [9]:
# zx = np.random.randint(1,100,(11,3))
# print(zx)
# mins_new, ranges_new = saturation_sort(zx)
# print(mins_new, ranges_new)

In [10]:
def _scalar_quantize(A, proportion = 0.3, axis=0, signed=False, nbits=8):
    unsigned_maxval = float(1 << int(nbits)) - 1

    # # TODO rm
    # # return np.zeros((A.shape[0], 1)), np.ones((A.shape[0], 1)), A
    # # offsets = np.zeros((A.shape[0], 1))
    # offsets = A.min(axis=1, keepdims=True)
    # # scales = maxval / np.ones((A.shape[0], 1))
    # scales = maxval / A.max(axis=1, keepdims=True)
    # Aq = (A - offsets) * scales
    # return offsets, scales, Aq
    mins, ranges = saturation_sort(A, proportion = proportion, axis = axis)
    # maxval = float(1 << int(nbits)) - 1
    # mins = A.min(axis=axis, keepdims=True)
    # A_offset = A - offsets
    # ranges = (A - mins).max(axis=axis, keepdims=True) + 1e-20
    # print("ranges:")
    # print(ranges)
    scales = unsigned_maxval / ranges
    # Aq = (A_offset * (maxval / scales)).astype(np.int)
    # Aq = (A_offset * scales).astype(np.int)

    if signed:
        # sign_offset = 1 << (nbits - 1)  # 8 bits -> 128
        # A_offset -= sign_offset
        offsets = mins + (ranges * (128. / 255))
        minval = -(1 << (nbits - 1))
        maxval = -minval - 1
    else:
        offsets = mins
        minval = 0
        maxval = (1 << nbits) - 1

    Aq = (A - offsets) * scales
    # print("min, max A:", Aq.min(), Aq.max())  # looks good
    Aq = np.clip(Aq, minval, maxval).astype(int)

    return offsets, scales, Aq, minval, maxval

In [11]:
def test_user_quantize(A, B, offsets, scales, minval, maxval, axis=0, signed=False, nbits=8):
    Aq = (A - offsets) * scales
    Aq = np.clip(Aq, minval, maxval).astype(int) # 量化后的值（整数
    Aqreal = Aq / scales + offsets # 量化后对应的实际值
    print("A:")
    print(A)
    print("Aq:")
    print(Aq)
    print("Aqreal")
    print(Aqreal)
    C = np.matmul(Aqreal, B)

    return C

In [12]:
def test_user_scalar_quantize(A, B, A_offsets, A_scales, A_minval, A_maxval, B_offsets, B_scales, B_minval, B_maxval):
    Aq = (A - A_offsets) * A_scales
    Aq = np.clip(Aq, A_minval, A_maxval).astype(int) # 量化后的值（整数
    Aqreal = Aq / A_scales + A_offsets # 量化后对应的实际值

    Bq = (B - B_offsets) * B_scales
    Bq = np.clip(Bq, B_minval, B_maxval).astype(int) # 量化后的值（整数
    Bqreal = Bq / B_scales + B_offsets # 量化后对应的实际值

    # print("A:")
    # print(A)
    # print("Aq:")
    # print(Aq)
    # print("Aqreal")
    # print(Aqreal)
    C = np.matmul(Aqreal, Bqreal)

    return C

In [13]:
# ran_mat = np.rint(10*np.random.random(size=(5,5)))
# print(ran_mat)

In [14]:
# offsets, scales, Aq, minval, maxval = _scalar_quantize(ran_mat, proportion=prop, axis=0, signed=False, nbits=8)

In [15]:
# print(offsets)
# print(scales)
# print(Aq)
# print(minval)
# print(maxval)

In [16]:
data_to_fc_train = np.load(os.path.join(dir_train, data_to_fcpath_train))
print(data_to_fc_train.shape)
weight = np.load(os.path.join(dir_train, weightpath))
print(weight.shape)
data_to_fc_test = np.load(os.path.join(dir_test, data_to_fcpath_test))
print(data_to_fc_test.shape)
bias = np.load(dir_train+'/'+biaspath)

(96000, 2048)
(2048, 256)
(32000, 2048)


In [17]:
train_offsets, train_scales, train_Aq, train_minval, train_maxval = _scalar_quantize(data_to_fc_train, proportion=prop_A, axis=0, signed=False, nbits=qbits)

In [18]:
if user_method == "METHOD_USER_QUANTIZE":
    y_out_matmul = test_user_quantize(data_to_fc_test, weight, train_offsets, train_scales, train_minval, train_maxval)
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [19]:
if user_method == "METHOD_USER_QUANTIZE":
    print(y_out_last)
    print(y_out_last.dtype)
    print(y_out_last.shape)
    np.save(dir_result+'/'+'UserQuantize'+'fc1_prop%0.3f_fb%i_qb%i.npy' % (prop, feedback_bits, qbits), y_out_last.astype(np.float32))


In [20]:
# print(train_offsets.shape)
# print(train_scales.shape)
# print(train_Aq.shape)

In [21]:
# print(train_offsets)

In [22]:
if user_method == "METHOD_USER_SCALAR_QUANTIZE":
    weight_offsets, weight_scales, weight_Aq, weight_minval, weight_maxval = _scalar_quantize(weight, proportion=prop_B, axis=1, signed=False, nbits=qbits)
    y_out_matmul = test_user_scalar_quantize(data_to_fc_test, weight, train_offsets, train_scales, train_minval, train_maxval, weight_offsets, weight_scales, weight_minval, weight_maxval)
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [23]:
if user_method == "METHOD_USER_SCALAR_QUANTIZE":
    print(y_out_last)
    print(y_out_last.dtype)
    print(y_out_last.shape)
    np.save(dir_result+'/'+'UserScalarQuantize'+'fc1_propA%0.3f_propB%0.3f_fb%i_qb%i.npy' % (prop_A, prop_B, feedback_bits, qbits), y_out_last.astype(np.float32))

[[-0.08589713  0.01957862 -0.54318082 ... -0.07518468  0.08027018
   0.00573426]
 [-0.23335717 -0.12642624 -0.29166916 ... -0.37956292  0.01083382
  -0.07693753]
 [ 0.39297602 -0.04998817 -0.23789372 ... -0.50742883  0.18774957
   0.11985465]
 ...
 [ 0.35143678  0.1430799  -0.52140565 ... -0.17485511 -0.2048602
   0.00773251]
 [ 0.24491932  0.01230494 -0.24259499 ...  0.05879867 -0.00268466
   0.05266514]
 [-0.30797064  0.11154683 -0.42690159 ... -0.34412207 -0.14496699
   0.17692375]]
float64
(32000, 256)
